In [1]:
!hdfs dfs -D dfs.replication=1 -cp -f data/*.jsonl hdfs://nn:9000/
!hdfs dfs -D dfs.replication=1 -cp -f data/*.csv hdfs://nn:9000/


In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "1G")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 14:40:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
problems_df = spark.read.json("hdfs://nn:9000/problems.jsonl")

In [4]:
problems_df.limit(5).show()


+-------------+--------+---------+---------+---------------+----------+---------------+-------------------------+------------------+--------------------+-------------+----------+------------+------+----------+
|cf_contest_id|cf_index|cf_points|cf_rating|        cf_tags|difficulty|generated_tests|is_description_translated|memory_limit_bytes|                name|private_tests|problem_id|public_tests|source|time_limit|
+-------------+--------+---------+---------+---------------+----------+---------------+-------------------------+------------------+--------------------+-------------+----------+------------+------+----------+
|          322|       A|    500.0|     1000|            [0]|         7|             93|                    false|         256000000|322_A. Ciel and D...|           45|         1|           2|     2|         1|
|          760|       D|   1000.0|     1600|         [1, 2]|        10|             51|                    false|         256000000|  760_D. Travel Card|       

In [5]:
type(problems_df)

pyspark.sql.dataframe.DataFrame

In [6]:
# problems_df.limit(1).show()

In [7]:
problems_df=spark.read.format("JSON").load("hdfs://nn:9000/problems.jsonl")

In [8]:
problems_df.limit(5).show()

+-------------+--------+---------+---------+---------------+----------+---------------+-------------------------+------------------+--------------------+-------------+----------+------------+------+----------+
|cf_contest_id|cf_index|cf_points|cf_rating|        cf_tags|difficulty|generated_tests|is_description_translated|memory_limit_bytes|                name|private_tests|problem_id|public_tests|source|time_limit|
+-------------+--------+---------+---------+---------------+----------+---------------+-------------------------+------------------+--------------------+-------------+----------+------------+------+----------+
|          322|       A|    500.0|     1000|            [0]|         7|             93|                    false|         256000000|322_A. Ciel and D...|           45|         1|           2|     2|         1|
|          760|       D|   1000.0|     1600|         [1, 2]|        10|             51|                    false|         256000000|  760_D. Travel Card|       

In [9]:
type(problems_df)


pyspark.sql.dataframe.DataFrame

In [10]:
problems_df.rdd


MapPartitionsRDD[20] at javaToPython at NativeMethodAccessorImpl.java:0

In [11]:
#q1
filter_count = problems_df.rdd.filter(lambda row: row.cf_rating >= 1600 and row.private_tests and "_A." in row.name ).count()
filter_count

217

In [12]:
#q2
from pyspark.sql.functions import col, expr
problems_df.filter(expr("cf_rating >=1600")).filter(expr("private_tests > 0")).filter(expr("contains(name,'_A.')")).count()


217

In [13]:
problems_df.write.mode("overwrite").saveAsTable("problems")

25/03/20 14:40:48 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/03/20 14:40:48 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/03/20 14:40:51 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/03/20 14:40:51 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.4
25/03/20 14:40:52 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/03/20 14:40:55 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/03/20 14:40:55 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/03/20 14:40:55 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/03/20 14:40:55 W

In [14]:
df = spark.read.table("problems")
type(df)

pyspark.sql.dataframe.DataFrame

In [15]:
#q3
spark.sql("""
SELECT *
FROM problems
WHERE cf_rating >= 1600 
AND private_tests > 0
AND contains(name,'_A.')
""").count()

217

In [16]:
#q4
solutions_df=spark.read.format("JSON").load("hdfs://nn:9000/solutions.jsonl")
(
    solutions_df
    .write
    .bucketBy(4, "language")
    .mode("overwrite")
    .saveAsTable("solutions")
)
spark.sql("""
SELECT language, COUNT(*)
FROM solutions
GROUP BY language
""").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[language#428], functions=[count(1)])
   +- HashAggregate(keys=[language#428], functions=[partial_count(1)])
      +- FileScan parquet spark_catalog.default.solutions[language#428] Batched: true, Bucketed: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[hdfs://nn:9000/user/hive/warehouse/solutions], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<language:string>, SelectedBucketsCount: 4 out of 4




In [17]:
#q5
import pandas as pd
csvs = ["languages", "problem_tests", "sources", "tags"]
for csv in csvs:
    table = spark.read.format("csv").option("header", True).load(f"hdfs://nn:9000/{csv}.csv")
    table.createOrReplaceTempView(csv)

ans = spark.sql("SHOW TABLES").toPandas()
#copied from "pandas use column values as key as value" search ai
ans.set_index('tableName')['isTemporary'].to_dict()

{'problems': False,
 'solutions': False,
 'languages': True,
 'problem_tests': True,
 'sources': True,
 'tags': True}

In [18]:
#q6
'''Join the solutions table with the problems table using an inner join on the problem_id column. 
Note that the source column in problems is an integer. Join this column with the source_id column 
from the sources CSV. Find the number of correct PYTHON3 solutions from CODEFORCES.
'''
spark.sql("""
SELECT *
FROM solutions
INNER JOIN problems ON solutions.problem_id = problems.problem_id
INNER JOIN sources ON problems.source = sources.source
WHERE is_correct = true AND solutions.language = "PYTHON3" AND sources.source_name = "CODEFORCES"
""").count()

10576

In [19]:
# spark.sql("""
# SELECT *
# FROM sources
# """).limit(3).show()

In [20]:
# spark.sql("""
# SELECT *
# FROM solutions
# """).limit(3).show()

In [21]:
#q7
'''
<= 5 is Easy
<= 10 is Medium
Otherwise Hard
'''
difficulties = spark.sql("""
SELECT difficulty,
CASE 
    WHEN difficulty <= 5 THEN 'Easy'
    WHEN difficulty <= 10 THEN 'Medium'
    ELSE 'Hard'
END AS levels
FROM problems
""").toPandas()
counts = difficulties['levels'].value_counts().to_dict()
counts

{'Medium': 5768, 'Hard': 2396, 'Easy': 409}

In [34]:
#q8
from pyspark.sql.functions import mean

problemTests = spark.sql("""
SELECT *
FROM problem_tests
WHERE is_generated = false
""")
import time
ts = []
t0 = time.time()
problemTests.select(mean(col("input_chars")))
problemTests.select(mean(col("output_chars")))
t1 = time.time()
time1 = t1-t0
# print(time)
ts.append(time1)
newProblemTests = problemTests.cache()
t2 = time.time()
newProblemTests.select(mean(col("input_chars")))
newProblemTests.select(mean(col("output_chars")))
t3 = time.time()
time2 = t3-t2
# print(time)
ts.append(time2)
t4 = time.time()
newProblemTests.select(mean(col("input_chars")))
newProblemTests.select(mean(col("output_chars")))
t5 = time.time()
time3 = t5-t4
# print(time)
ts.append(time3)
problemTests.unpersist()
ts

[0.028772830963134766, 0.02396845817565918, 0.024501323699951172]

In [53]:
'''
train dataset: cf_rating is >0, and problem_id in an EVEN number
test dataset: cf_rating is >0, and problem_id in an ODD number
missing dataset: cf_rating is 0
'''
train = spark.sql("""
SELECT sources.source, problems.source, source_name, problem_id, cf_rating, difficulty, time_limit, memory_limit_bytes
FROM problems
INNER JOIN sources ON problems.source = sources.source
WHERE sources.source_name = "CODEFORCES" and cf_rating > 0 and problem_id % 2 = 0
""")
test = spark.sql("""
SELECT sources.source, problems.source, source_name, problem_id, cf_rating, difficulty, time_limit, memory_limit_bytes
FROM problems
INNER JOIN sources ON problems.source = sources.source
WHERE sources.source_name = "CODEFORCES" and cf_rating > 0 and problem_id % 2 = 1
""")
missing = spark.sql("""
SELECT sources.source, problems.source, source_name, problem_id, cf_rating, difficulty, time_limit, memory_limit_bytes
FROM problems
INNER JOIN sources ON problems.source = sources.source
WHERE sources.source_name = "CODEFORCES" and cf_rating = 0
""")

In [60]:
#q9
from pyspark.ml.regression import DecisionTreeRegressor       
from pyspark.ml.regression import DecisionTreeRegressionModel  
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline     
from pyspark.ml.pipeline import PipelineModel 
from pyspark.ml.evaluation import RegressionEvaluator

va = VectorAssembler(inputCols=["difficulty", "time_limit", "memory_limit_bytes"], outputCol="features")
# va.transform(train).limit(5).show()
dt = DecisionTreeRegressor(featuresCol="features", labelCol="cf_rating")
pipe = Pipeline(stages=[va, dt])
model = pipe.fit(train)
r2score = RegressionEvaluator(predictionCol="prediction", labelCol="cf_rating", metricName="r2")
r2score.evaluate(model.transform(test))

0.5929835263198762

In [94]:
#q10
avgs = []
avgTrain = train.select(mean(col("cf_rating")))
#copied from https://stackoverflow.com/questions/31961971/spark-extracting-single-value-from-dataframe
avgs.append(avgTrain.head()[0])
avgTest = test.select(mean(col("cf_rating")))
avgs.append(avgTest.head()[0])
# avgs.append(avgTrain)


predMissing = model.transform(missing)
avgMissing = predMissing.select(mean(col("prediction")))
avgs.append(avgMissing.head()[0])
tuple(avgs)

(1887.9377431906614, 1893.1106471816283, 1950.4728638818783)